In [1]:
import untangle
import pandas as pd

## Melee

In [55]:
def parse_xml(fp: str) -> pd.DataFrame:
    xml = untangle.parse(fp)
    data = []
    for thingdef in xml.Defs.ThingDef:
        try:
            name = thingdef.defName.cdata
        except AttributeError:
            continue
        data.append(
            {
                "name": name,
                "attacks": [parse_attack(li) for li in thingdef.tools.li]
            }
        )
    df = pd.DataFrame(data)
    df["dps"] = df.attacks.apply(lambda x: calculate_dps(x))
    df["ap"] = [max([attack["armorPenetration"] for attack in attacks]) * 100 for attacks in df.attacks]
    df.set_index("name", inplace=True)
    #df.drop(columns="attacks", inplace=True)
    return df

In [46]:
def parse_attack(li: untangle.Element):
    attack = {}
    attack["damage"] = float(li.power.cdata)
    attack["cooldown"] = float(li.cooldownTime.cdata)
    if li.get_elements("armorPenetration"):
        attack["armorPenetration"] = float(li.armorPenetration.cdata)
    else:
        attack["armorPenetration"] = attack["damage"] * 0.015
    return attack

In [47]:
def calculate_dps(attacks: list) -> float:
    weights = [attack["damage"] * attack["damage"] for attack in attacks]
    chances = [weight / sum(weights) for weight in weights]
    avg_dmg = sum([attacks[i]["damage"] * chances[i] for i in range(len(attacks))])
    avg_cooldown = sum([attacks[i]["cooldown"] * chances[i] for i in range(len(attacks))])
    return round(avg_dmg / avg_cooldown, 2)

In [56]:
medieval = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Core/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
neolithic = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Core/Defs/ThingDefs_Misc/Weapons/MeleeNeolithic.xml")
medieval_ideology = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Royalty/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
ultratech = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Royalty/Defs/ThingDefs_Misc/Weapons/MeleeUltratech.xml")
bladelink = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Royalty/Defs/ThingDefs_Misc/Weapons/MeleeBladelink.xml")

In [49]:
melee = pd.concat((medieval, neolithic, medieval_ideology, ultratech, bladelink))

In [50]:
melee.sort_values(by="dps", ascending=False)

,dps,ap
name,,
MeleeWeapon_MonoSwordBladelink,16.03,90.00
MeleeWeapon_ZeusHammerBladelink,13.40,46.50
MeleeWeapon_MonoSword,12.08,90.00
MeleeWeapon_PlasmaSwordBladelink,11.11,34.50
MeleeWeapon_Zeushammer,9.95,46.50
MeleeWeapon_LongSword,8.60,34.50
MeleeWeapon_Spear,7.91,50.00
MeleeWeapon_PlasmaSword,7.85,31.50
MeleeWeapon_Gladius,7.52,24.00


In [51]:
vwe_industrial = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814383360/1.3/Defs/ThingDefs_Misc/Weapons/MeleeIndustrial.xml")
vwe_medieval = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814383360/1.3/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
vwe_neolithic = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814383360/1.3/Defs/ThingDefs_Misc/Weapons/MeleeNeolithic.xml")
vwe_tribal_neolithic = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/2454918552/1.3/Defs/ThingDefs_Misc/MeleeNeolithic.xml")
vwe_viking_medieval = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/2231295285/1.3/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
vwe_viking_ultra = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/2231295285/1.3/Defs/ThingDefs_Misc/Weapons/MeleeUltra.xml")

In [52]:
melee = pd.concat((melee, vwe_industrial, vwe_medieval, vwe_neolithic, vwe_tribal_neolithic, vwe_viking_medieval, vwe_viking_ultra))

In [60]:
melee.sort_values(by="dps", ascending=False)

,dps,ap
name,,
MeleeWeapon_MonoSwordBladelink,16.03,90.00
MeleeWeapon_ZeusHammerBladelink,13.40,46.50
MeleeWeapon_MonoSword,12.08,90.00
MeleeWeapon_PlasmaSwordBladelink,11.11,34.50
VFEV_CryptoHeavyAxe,10.89,35.00
MeleeWeapon_Zeushammer,9.95,46.50
VWE_MeleeWeapon_Halberd,9.23,70.00
MeleeWeapon_LongSword,8.60,34.50
VWE_MeleeWeapon_BattleAxe,8.43,15.00


* Halberds have too much Armor Penetration
* Combat knifes and Shivs have too high DPS
* Shovels have more DPS than Dane Axe
* Wrench and Seax should be last
* Combat Knife and Shiv have higher AP than Battle Axe and more DPS than Gladius / DaneAxe ?

## Armor

In [80]:
calculate_dps([{"damage": 8, "cooldown": 2}, {"damage": 12, "cooldown": 2}])

5.38